In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset, Subset
from torch import nn
from transformers import BertTokenizer, BertModel, BertPreTrainedModel, BertConfig
from torch.optim import AdamW
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from tqdm import tqdm
import json

c:\Users\wyatt\anaconda3\envs\hackGT\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and tokenize data

In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
max_length = 128

def load_data(csv_file):
    df = pd.read_csv(csv_file)
    texts = df["text"].tolist()

    for text in texts:
        if type(text) != type("test"):
            print(text, type(text))
    
    labels = torch.tensor(df["label"].tolist(), dtype=torch.float)

    encodings = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    dataset = TensorDataset(encodings["input_ids"], encodings["attention_mask"], labels)
    return dataset

dataset = load_data("fineTuning3.csv")  # single CSV with all samples

## Define regression model

In [3]:
class BertForRegression(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.regressor = nn.Linear(config.hidden_size, 1)
        self.init_weights()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        return self.regressor(pooled_output).squeeze(-1)

## Train/test split (80/20) + K-Fold CV

In [4]:
def run_training(dataset, k_folds=5, num_epochs=3, batch_size=16):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    fold_results = []

    for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
        print(f"\n----- Fold {fold+1} / {k_folds} -----")

        train_subsampler = Subset(dataset, train_ids)
        test_subsampler = Subset(dataset, test_ids)

        train_loader = DataLoader(train_subsampler, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_subsampler, batch_size=batch_size)

        config = BertConfig.from_pretrained("bert-base-uncased")
        model = BertForRegression.from_pretrained("bert-base-uncased", config=config).to(device)
        optimizer = AdamW(model.parameters(), lr=2e-5)
        criterion = nn.MSELoss()

        # Training
        for epoch in range(num_epochs):
            model.train()
            total_loss = 0
            for input_ids, attention_mask, labels in tqdm(train_loader, desc=f"Fold {fold+1} Epoch {epoch+1}"):
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
            print(f"Fold {fold+1} Epoch {epoch+1} Train Loss: {total_loss/len(train_loader):.4f}")

        # Evaluation
        model.eval()
        mse_total = 0
        with torch.no_grad():
            for input_ids, attention_mask, labels in test_loader:
                input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
                outputs = model(input_ids, attention_mask)
                mse_total += criterion(outputs, labels).item() * input_ids.size(0)

        mse_total /= len(test_subsampler)
        fold_results.append(mse_total)
        print(f"Fold {fold+1} Test MSE: {mse_total:.4f}")

    avg_mse = sum(fold_results) / len(fold_results)
    print(f"\n==== Cross-validation MSE across {k_folds} folds: {avg_mse:.4f} ====")

    # Save last model + tokenizer
    model.save_pretrained("./bert_sentiment_regression")
    tokenizer.save_pretrained("./bert_sentiment_regression")

#run_training(dataset, k_folds=5, num_epochs=5)

## Test

In [5]:
model = BertForRegression.from_pretrained("./bert_sentiment_regression")
tokenizer = BertTokenizer.from_pretrained("./bert_sentiment_regression")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Load data from full.json instead of testTweets.txt
with open("./gtvwoke.json", "r") as file:
    data = json.load(file)

# List to store results for export
results = []

# Process each entry in the JSON data
for entry in data:
    text = entry["body_html"]
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    
    # Keep only input_ids and attention_mask
    inputs = {k: v.to(device) for k, v in inputs.items() if k in ["input_ids", "attention_mask"]}
    
    with torch.no_grad():
        prediction = model(**inputs)
    
    prediction_score = prediction.item()/8
    
    # Store result for export
    result_entry = {
        "text": text,
        "prediction": prediction_score,
        "timestamp": entry["created_utc"]
    }
    results.append(result_entry)
    
    print(f"Text: {text}\nPrediction: {prediction_score:0.3f}\nTimestamp: {entry['created_utc']}\n")

# Export results to JSON file
with open("./testResults2.json", "w") as output_file:
    json.dump(results, output_file, indent=2)

print(f"Results exported to testResults.json with {len(results)} entries.")

Text: UGA fans in here crying for what
Prediction: 0.317
Timestamp: 1759004014.0

Text: Idk about those guys. I’m glad yall won. I want our game to mean something
Prediction: 0.852
Timestamp: 1759004440.0

Text: We never pull for teck, don't you know?
Prediction: 0.421
Timestamp: 1759005603.0

Text: If they’re ranked #1 when we beat them at the end of the season it’s good for us
Prediction: 0.841
Timestamp: 1759006783.0

Text: TO HELL WITH gEORGIA
Prediction: 0.207
Timestamp: 1759004350.0

Text: Lmfao at uga fans of all people complaining about refereeing  Fuck yourselves to the moon, guys
Prediction: 0.132
Timestamp: 1759004226.0

Text: 9-0 dude. see you in 10 years.
Prediction: 0.838
Timestamp: 1759005964.0

Text: Both of your flairs are slaves to us lmao
Prediction: 0.449
Timestamp: 1759004501.0

Text: Dos Equis made a call to have WF “go for dos”. Cost them the game
Prediction: 0.229
Timestamp: 1759003879.0

Text: Oh. I noticed that too. I thought that ads showed up AFTER they anno